In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/for-sentiment-training/to_train_senti.csv
/kaggle/input/for-sentiment-training/to_test_senti.csv
/kaggle/input/bert-base-uncased/vocab.txt
/kaggle/input/bert-base-uncased/pytorch_model.bin
/kaggle/input/bert-base-uncased/config.json


In [2]:
train = pd.read_csv("../input/for-sentiment-training/to_train_senti.csv")
test = pd.read_csv("../input/for-sentiment-training/to_test_senti.csv")

In [3]:
test.head()

,id,text,target_entity,label,sentiment
0,153,LOCATION1 is in Greater London and is a ve...,LOCATION1,safety,Positive
1,1130,All the neighborhoods around LOCATION1 are v...,LOCATION1,general,Positive
2,1130,All the neighborhoods around LOCATION1 are v...,LOCATION1,safety,Positive
3,1271,"Cheap is LOCATION2, LOCATION1, but not reall...",LOCATION1,general,Negative
4,1271,"Cheap is LOCATION2, LOCATION1, but not reall...",LOCATION1,price,Positive


In [4]:
train.head()

,id,text,target_entity,label,sentiment
0,1430,LOCATION1 is transforming and the prices w...,LOCATION1,price,Negative
1,1244,And LOCATION1 is ten mins direct on the tube...,LOCATION1,transit-location,Positive
2,209,Another option is LOCATION1 which is very ce...,LOCATION1,transit-location,Positive
3,2824,Best bet is around LOCATION2 and LOCATION1 a...,LOCATION1,general,Positive
4,2824,Best bet is around LOCATION2 and LOCATION1 a...,LOCATION2,general,Positive


In [5]:
import transformers

MAX_LEN = 512
TRAIN_BATCH_SIZE = 12
VALID_BATCH_SIZE = 12
EPOCHS = 10
BERT_PATH = "../input/bert-base-uncased/"
MODEL_PATH = "model.bin"
TRAINING_FILE = "../input/absa-dataset/input/bert-pair/train_NLI_M.tsv"
DEV_FILE = "../input/absa-dataset/input/bert-pair/dev_NLI_M.tsv"
TEST_FILE = "../input/absa-dataset/input/bert-pair/test_NLI_M.tsv"
TOKENIZER = transformers.BertTokenizer.from_pretrained(BERT_PATH, do_lower_case=True)

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [6]:
import torch

class BERTDataset:
    def __init__(self, sentence1s,targets):
        self.sentence1s = sentence1s
        #self.sentence2s = sentence2s
        self.targets = targets
        self.tokenizer = TOKENIZER
        self.max_len = MAX_LEN
        
    def __len__(self):
        return len(self.sentence1s)
                   
        
    def __getitem__(self, item):
        sentence1 = str(self.sentence1s[item])
        sentence1 = " ".join(sentence1.split())
        
        #sentence2 = str(self.sentence2s[item])
        #sentence2 = " ".join(sentence2.split())

        inputs = self.tokenizer.encode_plus(sentence1,
                                            None,
                                            add_special_tokens=True, 
                                            max_length=self.max_len,
                                            pad_to_max_length=True,
                                           )

        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]
        token_type_ids = inputs["token_type_ids"]
        
        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            "targets": torch.tensor(self.targets[item], dtype=torch.long),
        }


In [7]:
import transformers
import torch.nn as nn


class BERTBaseUncased(nn.Module):
    def __init__(self):
        super(BERTBaseUncased, self).__init__()
        self.bert = transformers.BertModel.from_pretrained(BERT_PATH)
        self.bert_drop = nn.Dropout(0.3)
        self.out = nn.Linear(768, 3)

    def forward(self, ids, mask, token_type_ids):
        _, o2 = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids)
        bo = self.bert_drop(o2)
        output = self.out(bo)
        return output


In [8]:
import torch
import torch.nn as nn
from tqdm import tqdm
import pdb


def loss_fn(outputs, targets):
    #targets = torch.argmax(targets, 1)
    #loss = nn.CrossEntropyLoss()(outputs, targets.view(-1, 1))
    loss = nn.CrossEntropyLoss()(outputs, targets)
    
    return loss


def train_fn(data_loader, model, optimizer, device, scheduler):
    model.train()

    for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
        ids = d["ids"]
        token_type_ids = d["token_type_ids"]
        mask = d["mask"]
        targets = d["targets"]

        ids = ids.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        #pdb.set_trace()
        targets = targets.to(device, dtype=torch.long)
        #pdb.set_trace()

        optimizer.zero_grad()
        outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()


def eval_fn(data_loader, model, device):
    model.eval()
    fin_targets = []
    fin_outputs = []
    with torch.no_grad():
        for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
            ids = d["ids"]
            token_type_ids = d["token_type_ids"]
            mask = d["mask"]
            targets = d["targets"]

            ids = ids.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.long)

            outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
            act = nn.Softmax(dim=1)
            outputs = act(outputs)
            fin_targets.extend(targets.detach().cpu().numpy().tolist())
            fin_outputs.extend(outputs.cpu().numpy().tolist())
            
    return fin_outputs, fin_targets

In [9]:
import pandas as pd
import torch.nn as nn
import numpy as np

#from model import BERTBaseUncased
from sklearn import model_selection
from sklearn import metrics
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn import preprocessing

def run():
    
    encoder = preprocessing.LabelEncoder()
    train.loc[:, "sentiment"] = encoder.fit_transform(train["sentiment"])
    #dev_df.loc[:, "label"] = encoder.transform(dev_df["label"])
    test.loc[:, "sentiment"] = encoder.transform(test["sentiment"])
    
    
    train_dataset = BERTDataset(sentence1s = train.text.values, targets=train.sentiment.values)
    
    train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, num_workers=4)
    
    valid_dataset = BERTDataset(sentence1s = test.text.values,targets=test.sentiment.values)
    
    valid_data_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=VALID_BATCH_SIZE, num_workers=4)
    
    device = torch.device("cuda")
    model = BERTBaseUncased()
    model.to(device)
    
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [{"params": [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], "weight_decay": 0.001,},
                            {"params": [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], "weight_decay": 0.0,},]
    num_train_steps = int(len(train) / TRAIN_BATCH_SIZE * EPOCHS)
    optimizer = AdamW(optimizer_parameters, lr=3e-5)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_train_steps)
    
    model = nn.DataParallel(model)
    
    best_accuracy = 0
    
    for epoch in range(EPOCHS):
        print(epoch)
        train_fn(train_data_loader, model, optimizer, device, scheduler)
        outputs, targets = eval_fn(valid_data_loader, model, device)
        #pdb.set_trace()
        outputs = torch.tensor(outputs)
        outputs = torch.argmax(outputs, dim=1)
        #outputs = np.array(outputs) >= 0.5
        accuracy = metrics.accuracy_score(targets, outputs)
        print(f"Accuracy Score =", {accuracy})
        if accuracy > best_accuracy:
            torch.save(model.state_dict(), MODEL_PATH)
            best_accuracy = accuracy
            
if __name__ == "__main__":
    run()


0


100%|██████████| 102/102 [00:22<00:00,  4.62it/s]


Accuracy Score = {0.819078947368421}
1


100%|██████████| 102/102 [00:21<00:00,  4.64it/s]


Accuracy Score = {0.8470394736842105}
2


...

Accuracy Score = {0.8552631578947368}
8



100%|██████████| 102/102 [00:22<00:00,  4.62it/s]

Accuracy Score = {0.8569078947368421}
9



100%|██████████| 102/102 [00:22<00:00,  4.61it/s]


Accuracy Score = {0.8585526315789473}


In [10]:
import torch
import pandas as pd
import torch.nn as nn
import numpy as np

#from model import BERTBaseUncased
from sklearn import model_selection
from sklearn import metrics
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn import preprocessing

def prediction():
    encoder = preprocessing.LabelEncoder()
    train.loc[:, "sentiment"] = encoder.fit_transform(train["sentiment"])
    test.loc[:, "sentiment"] = encoder.fit_transform(test["sentiment"])
    
    test_dataset = BERTDataset(sentence1s = test.text.values, targets=test.sentiment.values)
    
    test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=VALID_BATCH_SIZE, num_workers=4)
    
    device = torch.device("cuda")
    model = BERTBaseUncased()
    
    model = nn.DataParallel(model)
    model.load_state_dict(torch.load(MODEL_PATH))
    model.to(device)
    
    outputs, targets = eval_fn(test_data_loader, model, device)
    outputs = torch.tensor(outputs)
    outputs = torch.argmax(outputs, dim=1)
    out_df = pd.DataFrame({"id":test["id"].values})
    out_df['prediction'] = outputs
    out_df.to_csv("test_submission.csv", index=False)
    accuracy_test = metrics.accuracy_score(targets, outputs)
    print(f"Accuracy Score =", {accuracy_test})
    

In [11]:
prediction()


100%|██████████| 102/102 [00:22<00:00,  4.60it/s]


Accuracy Score = {0.8585526315789473}
